In [1]:
import pandas as pd
import numpy as np
import json
import re
from ast import literal_eval
from fuzzywuzzy import fuzz

In [2]:
data_path="../data/"
output_path="../outputs/"

In [3]:
tpredicted = [json.loads(line) for line in open('../../../test_outputs/sg_papers_scirex_predictions_final_3.jsonl')]
tdf_predicted = pd.DataFrame(tpredicted)
salient_test = [json.loads(line) for line in open('../../../test_outputs/salient_mentions_predictions_sg_papers_scirex_final_3.jsonl')]
df_salient_test = pd.DataFrame(salient_test)
sclusters = [json.loads(line) for line in open('../../../test_outputs/sg_papers_scirex_cluster_predictions_final_3.jsonl')]
tdf_sclusters = pd.DataFrame(sclusters)

df_salient_test=tdf_predicted.merge(df_salient_test,on='doc_id',how='left')
df_salient_test=df_salient_test.merge(tdf_sclusters,on='doc_id',how='left')



In [4]:
len(tpredicted)

8133

In [5]:
df_salient=df_salient_test.copy()

In [6]:
df_salient['Method']=[list() for _ in range(df_salient.shape[0])]
df_salient['Task']=[list() for _ in range(df_salient.shape[0])]
df_salient['Metric']=[list() for _ in range(df_salient.shape[0])]
df_salient['Dataset']=[list() for _ in range(df_salient.shape[0])]
df_salient['Material']=[list() for _ in range(df_salient.shape[0])]

In [7]:
df_salient=df_salient.assign(clusters=np.where(df_salient.clusters.isna(),None,df_salient.clusters))
df_salient=df_salient.assign(saliency=np.where(df_salient.saliency.isna(),None,df_salient.saliency))

In [8]:
df_salient.loc[(df_salient.saliency.isna()) & (df_salient.clusters.isna())]

,words,sections,sentences,ner,doc_id,saliency,spans,clusters,Method,Task,Metric,Dataset,Material
115,"[Breaking, the, quality]","[[0, 3]]","[[0, 3]]",[],farwell-1998-breaking,None,NaN,None,[],[],[],[],[]
199,"[Calls, for, Participation, Calls, for, Partic...","[[0, 32]]","[[0, 32]]",[],nn-1984-calls-participation,None,NaN,None,[],[],[],[],[]
473,"[A, Large, Rated, Lexicon, with, French, Medic...","[[0, 234]]","[[0, 39], [39, 58], [58, 85], [85, 103], [103,...",[],grabar-hamon-2016-large,None,NaN,None,[],[],[],[],[]
727,"[HimL, (Health, in, my]","[[0, 4]]","[[0, 4]]",[],haddow-2015-himl-health,None,NaN,None,[],[],[],[],[]
981,"[``A, pessimist, sees, the, difficulty, in, ev...","[[0, 140]]","[[0, 46], [46, 64], [64, 88], [88, 114], [114,...",[],kumar-etal-2017-pessimist,None,NaN,None,[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6940,"[Bilingual, Sign, Language, Dictionary, to, Le...","[[0, 15]]","[[0, 15]]",[],suzuki-etal-2004-bilingual,None,NaN,None,[],[],[],[],[]
7011,"[日本學生學習華語的聲調偏誤分析, :, 以二字調為例, (Tonal, errors, o...","[[0, 457]]","[[0, 389], [389, 457]]",[],zhang-chen-2005-ri,None,NaN,None,[],[],[],[],[]
7160,"[Catching, the, news, :, two, key, cases, from...","[[0, 111]]","[[0, 22], [22, 29], [29, 63], [63, 80], [80, 1...",[],margova-temnikova-2009-catching,None,NaN,None,[],[],[],[],[]
7320,"[Building, Renewable, Language, Assets, in, Go...","[[0, 6]]","[[0, 6]]",[],flaherty-johanson-2016-building,None,NaN,None,[],[],[],[],[]


In [9]:
lsaliency=[]
for i,row in df_salient.iterrows():
    #print(row['ner'])
    ner={}
    score={}
    saliency={}
    doc_dict={}
    ner_list=[]
    
    doc_ner={}
    for n in row['ner']:
        word=str.lower(' '.join(row['words'][n[0]:n[1]]))
        if n[2] in doc_ner:
            doc_ner[n[2]].append({'words':[word],'value':0,'top_word':word})
        else:
            doc_ner[n[2]]=list([{'words':[word],'value':0,'top_word':word}])
    if (row['clusters']!=None):
        for s,n in zip(row['saliency'],row['ner']):
            ner[str(n[0])+"_"+str(n[1])]=n[2]
            score[str(n[0])+"_"+str(n[1])]=s[3]
            saliency[str(n[0])+"_"+str(n[1])]=s[2]
        for k in row['clusters'].keys():
            scores=[]
            saliency_flag=0
            ner_list=[]
            word_list=[]
            for entity in row['clusters'][k]:
                #print((' '.join(row['words'][entity[0]:entity[1]])).lower())
                #print(ner[str(entity[0])+"_"+str(entity[1])])
                #print(score[str(entity[0])+"_"+str(entity[1])])
                ner_list.append(ner[str(entity[0])+"_"+str(entity[1])])
                scores.append(score[str(entity[0])+"_"+str(entity[1])])
                word=(' '.join(row['words'][entity[0]:entity[1]])).lower()
                word=re.sub('[^0-9a-zA-Z ]+', '', word)
                word=re.sub(' +', ' ',word)
                word_list.append(word)
                if saliency[str(entity[0])+"_"+str(entity[1])]==1:
                    saliency_flag=1
            #print(word_list)
            type_ent=set(ner_list).pop()
            if (type_ent in doc_dict) and (type_ent in ['Method','Task','Metric','Dataset']):
                doc_dict[type_ent].append({'words':word_list,'value':np.mean(scores),'top_word':word_list[scores.index(max(scores))]})
            else:
                doc_dict[type_ent]=list([{'words':word_list,'value':np.mean(scores),'top_word':word_list[scores.index(max(scores))]}])
    #print(doc_ner)
    #print()
    #print(doc_dict)
    #print("#########")
    
    for key in doc_ner.keys():
        if key not in doc_dict.keys():
            doc_dict[key]=doc_ner[key]
    for key in doc_dict.keys():
        df_salient.at[i,key]=doc_dict[key]
        
        #print()

In [10]:
df_labels_task=df_salient.loc[:,['doc_id','Task','Method']].rename(columns={'doc_id':'ID'})

In [11]:
df_labels_task=df_labels_task.rename(columns={'Task':'task_scirex','Method':'method_scirex'})

In [12]:
df_labels_task.loc[(df_labels_task.method_scirex.apply(lambda x:len(x)!=0))].shape

(7150, 3)

In [13]:
df_labels_task.loc[(df_labels_task.method_scirex.apply(lambda x:len(x)==0)) & (df_labels_task.task_scirex.apply(lambda x:len(x)==0))]

,ID,task_scirex,method_scirex
115,farwell-1998-breaking,[],[]
199,nn-1984-calls-participation,[],[]
473,grabar-hamon-2016-large,[],[]
727,haddow-2015-himl-health,[],[]
981,kumar-etal-2017-pessimist,[],[]
...,...,...,...
7058,price-etal-2020-six,[],[]
7160,margova-temnikova-2009-catching,[],[]
7231,casanellas-marg-2013-connectivity,[],[]
7320,flaherty-johanson-2016-building,[],[]


In [14]:
df_labels_task.loc[(df_labels_task.method_scirex.apply(lambda x:len(x)!=0)) | (df_labels_task.task_scirex.apply(lambda x:len(x)!=0))]

,ID,task_scirex,method_scirex
0,lin-etal-2006-generative,[{'words': ['natural language processing appli...,"[{'words': ['generative models', 'generative a..."
1,ghosh-etal-2020-iitp,"[{'words': ['offensive tweet identification', ...","[{'words': ['oth'], 'value': 0.080828607082366..."
2,grouin-2016-identification,"[{'words': ['ner', 'ner', 'ner'], 'value': 0.3...",[{'words': ['machinelearning and rule based sy...
3,volkova-etal-2014-inferring,[{'words': ['inferring user political preferen...,"[{'words': ['social media personal analytics',..."
4,rayner-etal-2003-limited,[{'words': ['limited domain english to japanes...,"[{'words': ['regulus 2', 'regulus 2 platform']..."
...,...,...,...
8128,landauer-2003-pasteurs,[{'words': ['optimal sequences of study materi...,"[{'words': ['lsa', 'lsa', 'lsa', 'lsa'], 'valu..."
8129,lee-etal-2020-counselling,"[{'words': ['virtual counsellor', 'counselling...","[{'words': ['word movers distance'], 'value': ..."
8130,widdows-etal-2003-unsupervised,[{'words': ['unsupervised monolingual and bili...,"[{'words': ['monolingual techniques', 'bilingu..."
8131,ye-etal-2018-interpretable,[{'words': ['text to text natural language gen...,"[{'words': ['sequenceto sequence model', 'seq2..."


In [15]:
df_labels_task.loc[(df_labels_task.method_scirex.apply(lambda x:len(x)!=0)) & (df_labels_task.task_scirex.apply(lambda x:len(x)!=0))]

,ID,task_scirex,method_scirex
0,lin-etal-2006-generative,[{'words': ['natural language processing appli...,"[{'words': ['generative models', 'generative a..."
1,ghosh-etal-2020-iitp,"[{'words': ['offensive tweet identification', ...","[{'words': ['oth'], 'value': 0.080828607082366..."
2,grouin-2016-identification,"[{'words': ['ner', 'ner', 'ner'], 'value': 0.3...",[{'words': ['machinelearning and rule based sy...
3,volkova-etal-2014-inferring,[{'words': ['inferring user political preferen...,"[{'words': ['social media personal analytics',..."
4,rayner-etal-2003-limited,[{'words': ['limited domain english to japanes...,"[{'words': ['regulus 2', 'regulus 2 platform']..."
...,...,...,...
8128,landauer-2003-pasteurs,[{'words': ['optimal sequences of study materi...,"[{'words': ['lsa', 'lsa', 'lsa', 'lsa'], 'valu..."
8129,lee-etal-2020-counselling,"[{'words': ['virtual counsellor', 'counselling...","[{'words': ['word movers distance'], 'value': ..."
8130,widdows-etal-2003-unsupervised,[{'words': ['unsupervised monolingual and bili...,"[{'words': ['monolingual techniques', 'bilingu..."
8131,ye-etal-2018-interpretable,[{'words': ['text to text natural language gen...,"[{'words': ['sequenceto sequence model', 'seq2..."


In [16]:
df_labels_task.to_json(output_path+"sg_ie/positives_tasks_methods_clusters_final_f.json")